In [ ]:
# !pip -q install sagemaker awscli boto3 --upgrade

In [4]:
import tarfile
import sagemaker
import tensorflow as tf
import tensorflow.keras as keras
import shutil
import os
import json
import numpy as np
import time

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

In [5]:
import boto3
client = boto3.client(service_name='sagemaker')
runtime = boto3.client(service_name='sagemaker-runtime')

## Download model and upload to S3

In [6]:
from tensorflow.keras.applications.resnet50 import ResNet50

def load_save_resnet50_model(model_path):
    model = ResNet50(weights='imagenet')
    shutil.rmtree(model_path, ignore_errors=True)
    model.save(model_path, include_optimizer=False, save_format='tf')

saved_model_dir = 'resnet50_saved_model' 
model_ver = '1'
model_path = os.path.join(saved_model_dir, model_ver)

load_save_resnet50_model(model_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: resnet50_saved_model/1/assets


In [7]:
shutil.rmtree('model.tar.gz', ignore_errors=True)
!tar cvfz model.tar.gz code -C resnet50_saved_model .

code/
code/requirements.txt
code/inference.py
./
./1/
./1/variables/
./1/variables/variables.data-00000-of-00001
./1/variables/variables.index
./1/saved_model.pb
./1/assets/


In [8]:
prefix = 'keras_models_serverless'
s3_model_path = sess.upload_data(path='model.tar.gz', key_prefix=prefix)

## Get model serving container

In [9]:
image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.1",
    py_version="py3",
    image_scope='inference',
    instance_type='ml.c5.large'
)

print('Container image with TensorFlow Serving:')
image_uri

Container image with TensorFlow Serving:


'763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-inference:2.1-cpu'

## 1. Create a model

In [10]:
from time import gmtime, strftime
model_name = 'keras-serverless' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Model name: ' + model_name)

create_model_response = client.create_model(
    ModelName = model_name,
    Containers=[{
        "Image": image_uri,
        "Mode": "SingleModel",
        "ModelDataUrl": s3_model_path,
    }],
    ExecutionRoleArn = role
)

print("Model Arn: " + create_model_response['ModelArn'])

Model name: keras-serverless2021-12-13-23-58-29
Model Arn: arn:aws:sagemaker:us-west-2:453691756499:model/keras-serverless2021-12-13-23-58-29


## 2. Define endpoint configuration

**MemorySizeInMB:** Choose any of the following values for your memory size: 1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB, or 6144 MB

**MaxConcurrency:** Choose between 1-50

In [11]:
keras_epc_name = "keras-serverless-epc" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=keras_epc_name,
    ProductionVariants=[
        {
            "VariantName": "kerasVariant",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 1,
            }
        },
    ],
)

print("Serverless Endpoint Configuration Arn: " + endpoint_config_response['EndpointConfigArn'])

Serverless Endpoint Configuration Arn: arn:aws:sagemaker:us-west-2:453691756499:endpoint-config/keras-serverless-epc2021-12-14-00-00-32


## 3. Create an endpoint

In [12]:
endpoint_name = "keras-serverless-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=keras_epc_name,
)

print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

Endpoint Arn: arn:aws:sagemaker:us-west-2:453691756499:endpoint/keras-serverless-ep2021-12-14-00-00-36


In [13]:
%%time
# wait for endpoint to reach a terminal state (InService) using describe endpoint
import time

describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
CPU times: user 173 ms, sys: 4.02 ms, total: 177 ms
Wall time: 3min 2s


### Test endpoint

In [14]:
file_name = 'kitten.jpg'

!wget -q https://s3.amazonaws.com/model-server/inputs/kitten.jpg -O {file_name}
with open(file_name, 'rb') as f:
    image_data = f.read()

In [18]:
%%time
response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='application/x-image', 
                                   Body=image_data)

pred = np.array(json.loads(response['Body'].read())['predictions'][0]).argsort()[-5:][::-1] 

CPU times: user 7 ms, sys: 0 ns, total: 7 ms
Wall time: 2.15 s


In [19]:
with open('imagenet_class_index.json', 'r') as f:
    labels = json.load(f)
    
for l in pred:
    print(labels[str(l)][1])

tiger_cat
lens_cap
Egyptian_cat
tabby
cougar


In [20]:
%%time
response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='application/x-image', 
                                   Body=image_data)

pred = np.array(json.loads(response['Body'].read())['predictions'][0]).argsort()[-5:][::-1] 

CPU times: user 23.3 ms, sys: 0 ns, total: 23.3 ms
Wall time: 2.15 s


In [21]:
keras_realtime_epc_name = "keras-realtime-epc" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=keras_realtime_epc_name,
    ProductionVariants=[
        {
            "VariantName": "kerasVariant",
            "ModelName": model_name,
            "InstanceType": "ml.c5.xlarge",
            "InitialInstanceCount": 1
        },
    ],
)

print("Realtime Endpoint Configuration Arn: " + endpoint_config_response['EndpointConfigArn'])

Realtime Endpoint Configuration Arn: arn:aws:sagemaker:us-west-2:453691756499:endpoint-config/keras-realtime-epc2021-12-14-00-08-19


In [22]:
realtime_endpoint_response = client.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=keras_realtime_epc_name)

In [23]:
%%time
# wait for endpoint to reach a terminal state (InService) using describe endpoint
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Updating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

Updating
Updating
Updating
Updating
Updating
InService
CPU times: user 79.3 ms, sys: 8.66 ms, total: 87.9 ms
Wall time: 1min 31s


In [27]:
%%time
response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='application/x-image', 
                                   Body=image_data)

pred = np.array(json.loads(response['Body'].read())['predictions'][0]).argsort()[-5:][::-1] 

CPU times: user 5.64 ms, sys: 0 ns, total: 5.64 ms
Wall time: 1.44 s
